# Sephora Skincare Products Scraper

This is a skincare product scraper that uses bs4 and regex to find and save product information into json.

In [1]:
# importing required packages
from bs4 import BeautifulSoup

import requests
import json
import re
import scrape_tools

In [2]:
def find_float(s):
    result = ''
    for i in s.split():
        try:
            #trying to convert i to float
            result = float(i)
            #break the loop if i is the first string that's successfully converted
            break
        except:
            continue
    return str(result)

In [3]:
def parse_product(product_url):
    product_page = requests.get(product_url)
    product_html = BeautifulSoup(product_page.content, 'lxml')

    skin_types = ['Normal', 'Oily', 'Combination', 'Dry', 'Sensitive']
    product_details = product_html.find('div', class_='css-pz80c5').get_text()

    product_brand = product_html.find('span', class_='css-euydo4').get_text()
    product_name = product_html.find('span', class_='css-0').get_text()
    product_price = product_html.find('div', class_='css-14hdny6').get_text()
    product_reviews = product_html.find('span', class_='css-2rg6q7').get_text()
    product_likes = product_html.find('span', {'data-at' : 'product_love_count'}).get_text()
    
    product_image = "https://www.sephora.com/" + product_html.find('image')['xlink:href']
    
    product_script = product_html.find_all('script')[5].contents[0]
    categories = re.findall('(?<="displayName":").*?(?=\")', product_script)
    product_category = categories[1]
    product_sub_category = categories[0]
    
    size_and_id = product_html.find('div', class_='css-1qf1va5').get_text()
    product_id = re.search('(?<=ITEM ).*', size_and_id)[0]
    
    product_rating = ''
    rating_text = product_html.find('div', class_='css-r17a09')['aria-label']
    rating_regex = re.search('^\D*(\d+)', rating_text)
    if rating_regex:
        product_rating = rating_regex[0]
    
    rating_script = re.search('(?<="productRating":).*?(?=\,)', product_script)
    if rating_script:
        product_rating = rating_script[0]
        
    product_skin_types = []
    for st in skin_types:
        if st in product_details[0:300]:
            product_skin_types.append(st)
            
    product_ingredients = ''
    if (len(product_html.find_all('div', class_='css-pz80c5')) > 2):
        product_ingredients = product_html.find_all('div', class_='css-pz80c5')[2].get_text()
        product_ingredients = product_ingredients.strip().replace(u'\xa0', u'') \
            .replace(u'\u2028', u'').replace(u'\r',u'').replace(u'\t',u'')
    return {
        'product_id' : product_id,
        'brand' : product_brand,
        'name' : product_name,
        'price' : product_price,
        'likes' : product_likes,
        'reviews': product_reviews,
        'rating' : product_rating,
        'product_url': product_url,
        'image_url' : product_image,
        'category' : product_category,
        'sub_category' : product_sub_category,
        'skin_types' : product_skin_types,
        'ingredients' : product_ingredients
    }

In [4]:
# testing parse function
url = 'https://www.sephora.com/shop/moisturizing-cream-oils-mists'
sephora_html = scrape_tools.get_html(url)
url1 = scrape_tools.find_product_urls(sephora_html)[1]
parse_product(url1)

{'product_id': '2025633',
 'brand': 'Drunk Elephant',
 'name': 'Protini™ Polypeptide Moisturizer',
 'price': '$68.00',
 'likes': '148509',
 'reviews': '2K reviews',
 'rating': '4.1548',
 'product_url': 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421',
 'image_url': 'https://www.sephora.com//productimages/sku/s2025633-main-Lhero.jpg',
 'category': 'Moisturizers',
 'sub_category': 'Moisturizers',
 'skin_types': ['Normal', 'Oily', 'Combination', 'Dry'],
 'ingredients': '-9 Signal Peptide Complex-Pygmy Waterlily Stem Cell Extract-Soybean Folic Acid Ferment Extract   Water, Dicaprylyl Carbonate, Glycerin, Cetearyl Alcohol, Cetearyl Olivate, Sorbitan Olivate, Sclerocarya Birrea Seed Oil, Bacillus/Soybean/ Folic Acid Ferment Extract, Nymphaea Alba Root Extract, sh-Oligopeptide-1, sh-Oligopeptide-2, sh-Polypeptide-1, sh-Polypeptide-9, sh-Polypeptide-11, Copper Palmitoyl Heptapeptide-14, Heptapeptide-15 Palmitate, Palmitoyl Tetrapeptide-7, Palmitoyl Tripeptide-1, Alanine, 

In [5]:
# categories to scrape on sephora

start_url = ['https://www.sephora.com/shop/moisturizing-cream-oils-mists',
            'https://www.sephora.com/shop/cleanser',
            'https://www.sephora.com/shop/facial-treatments',
            'https://www.sephora.com/shop/eye-treatment-dark-circle-treatment',
            'https://www.sephora.com/shop/face-mask',
            'https://www.sephora.com/shop/sunscreen-sun-protection',
            'https://www.sephora.com/shop/self-tanning-products',
            'https://www.sephora.com/shop/lip-treatments']

product_categories = [
    "Moisturizers",
    "Cleansers",
    "Facial Treatments",
    "Eye Treatments",
    "Face Masks",
    "Sunscreens",
    "Self Tanners",
    "Lip Treatments"
]

In [6]:
# get the number of pages
num_pages = []
for url in start_url:
    sephora_html = scrape_tools.get_html(url)
    num_pages.append(scrape_tools.find_num_pages(sephora_html))

In [ ]:
# check num pages
num_pages

[14, 9, 10, 4, 6, 4, 1, 3]

In [ ]:
all_products = {}
for i in range(len(product_categories)):
    print("scraping " + product_categories[i])
    url = start_url[i]
    product_list = {}
    for j in range(num_pages[i]):
        curr_url = url + '?currentPage=' + str(j + 1)
        page_html = scrape_tools.get_html(curr_url)
        product_urls = scrape_tools.find_product_urls(page_html)
        for k in range(len(product_urls)):
            pu = product_urls[k]
            product = parse_product(pu)
            product_id = product['product_id']
            del product['product_id']
            product_list[product_id] = product
        print("page " + str(j + 1) + " parse completed!")
    all_products[product_categories[i]] = product_list

scraping Moisturizers


In [21]:
skincare = {'skincare_products': all_products}

In [22]:
# save products to json
product_file = open('products.json', 'w')  

with open('products.json', 'w') as json_file:  
    json.dump(skincare, json_file)